# 📘 Agentic Architectures 1：Reflection

欢迎来到我们针对 21 个关键 agentic architectures 的系列中的第一个 notebook。本篇从最基础且强大的模式之一：**Reflection** 开始。

该模式让 Large Language Model（LLM）从简单的单次生成器，升级为更审慎且更稳健的推理者。与其给出想到的第一个答案，具备反思能力的 agent 会先退一步，对自己的输出进行批判、分析与改进。这样的自我迭代过程是构建更可靠、更高质量 AI 系统的基石。

### 定义
**Reflection** 架构让 agent 在返回最终答案之前，先对自己的输出进行批判与修订。它不再是单次生成，而是进行多步的内部独白：生成、评估、改进。这模拟了人类“起草—审阅—编辑”的过程，用以捕捉错误并提升质量。

### 高层工作流

1.  **Generate：** 根据用户提示生成初稿或解决方案。
2.  **Critique：** 转换成“评论者”角色，向自己发问：*"这份答案可能哪里有问题？"*、*"缺了什么？"*、*"是否最优？"*、或 *"是否存在逻辑漏洞或 bug？"*。
3.  **Refine：** 根据自我批判得到的洞见，生成最终的改进版本。

### 适用场景 / 应用
*   **代码生成：** 初版代码可能存在 bug、效率低或缺少注释。Reflection 让 agent 充当自我 code reviewer，在呈现最终脚本前修正错误、改进风格。
*   **复杂摘要：** 面对高密度文档，第一遍摘要可能遗漏细节或要点。反思步骤能帮助摘要更全面、更准确。
*   **创作与内容生成：** 邮件、博客、故事的首稿总是可以打磨。Reflection 能优化语气、清晰度与影响力。

### 优势与劣势
*   **优势：**
    *   **质量提升：** 直接针对并修正错误，输出更准确、稳健、推理更充分。
    *   **低门槛：** 概念简单，可用单个 LLM 实现，不依赖复杂外部工具。
*   **劣势：**
    *   **自我偏见：** agent 仍受自身知识与偏见限制。若它不知道更好的解法，就无法通过批判“发明”缺失的知识。它能修正“看得到”的问题，但不能凭空补足“看不到”的知识。
    *   **时延与成本增加：** 至少需要两次 LLM 调用（生成 + 批判/改进），比单次生成更慢、更贵。

## Phase 0：基础与环境搭建

在构建反思型 agent 之前，先完成环境设置：安装所需库、导入模块并配置 API keys。

### 步骤 0.1：安装核心库

**我们要做什么：**
将安装本项目所需的 Python 库。`langchain-nebius` 提供对 Nebius AI Studio 模型的访问，`langchain` 与 `langgraph` 提供核心编排框架，`python-dotenv` 管理我们的 API keys，而 `rich` 则用于更美观地打印输出。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

### 步骤 0.2：导入库并设置密钥

**我们要做什么：**
现在我们将导入所有必要的组件。通过 `python-dotenv` 从本地 `.env` 文件安全地加载 Nebius API key。我们还会设置 LangSmith 追踪，这对于调试多步 agentic 工作流非常关键。

**需要操作：** 你必须在与本 notebook 同目录下创建一个名为 `.env` 的文件，并像下面这样添加你的密钥：
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
```

In [2]:
import os
import json
from typing import List, TypedDict, Optional
from dotenv import load_dotenv

# Nebius and LangChain components
from langchain_nebius import ChatNebius
from pydantic import BaseModel, Field # Corrected import for Pydantic v2
from langgraph.graph import StateGraph, END

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.syntax import Syntax

# --- API Key and Tracing Setup ---
load_dotenv()

# Set up LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Reflection (Nebius)"

# Check that the keys are set
if not os.environ.get("NEBIUS_API_KEY"):
    print("NEBIUS_API_KEY not found. Please create a .env file and set it.")
if not os.environ.get("LANGCHAIN_API_KEY"):
    print("LANGCHAIN_API_KEY not found. Please create a .env file and set it for tracing.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1：构建 Reflection 的核心组件

一个健壮的 Reflection 架构不仅仅是一个简单的 prompt。我们将其搭建为结构化的三段式系统：**Generator**、**Critic** 和 **Refiner**。为确保可靠性，我们使用 Pydantic 模型为每一步定义预期的输出 schema。

### 步骤 1.1：用 Pydantic 定义数据 schema

**我们要做什么：**
我们将定义充当 LLM "契约" 的 Pydantic 模型。这样可以明确告诉 LLM 它的输出结构应该是什么——这对多步流程至关重要，因为某一步的输出将作为下一步的输入。

In [3]:
class DraftCode(BaseModel):
    """Schema for the initial code draft generated by the agent."""
    code: str = Field(description="The Python code generated to solve the user's request.")
    explanation: str = Field(description="A brief explanation of how the code works.")

class Critique(BaseModel):
    """Schema for the self-critique of the generated code."""
    has_errors: bool = Field(description="Does the code have any potential bugs or logical errors?")
    is_efficient: bool = Field(description="Is the code written in an efficient and optimal way?")
    suggested_improvements: List[str] = Field(description="Specific, actionable suggestions for improving the code.")
    critique_summary: str = Field(description="A summary of the critique.")

class RefinedCode(BaseModel):
    """Schema for the final, refined code after incorporating the critique."""
    refined_code: str = Field(description="The final, improved Python code.")
    refinement_summary: str = Field(description="A summary of the changes made based on the critique.")

print("Pydantic models for Draft, Critique, and RefinedCode have been defined.")

Pydantic models for Draft, Critique, and RefinedCode have been defined.


**输出讨论：**
我们已成功定义数据结构。`Critique` 模型尤为关键；通过要求 `has_errors` 与 `is_efficient` 等字段，我们引导 LLM 进行更结构化、更有用的评审，而不是仅仅让它 "review the code"。

### 步骤 1.2：初始化 Nebius LLM 与 Console

**我们要做什么：**
我们将初始化支持三种角色（Generator、Critic、Refiner）的 Nebius 语言模型。使用诸如 `meta-llama/Meta-Llama-3.1-8B-Instruct` 这样强大的模型，以确保各步骤的高质量推理。同时设置 `rich` console 用于整洁的格式化输出。

In [4]:
# Use a powerful Nebius model for generation and critique
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0.2)

# Initialize console for pretty printing
console = Console()

print("Nebius LLM and Console are initialized.")

Nebius LLM and Console are initialized.


### 步骤 1.3：创建 Generator 节点

**我们要做什么：**
该节点的唯一职责是根据用户请求生成第一版草稿。我们会将 `DraftCode` Pydantic 模型绑定到 Nebius LLM，以确保输出结构正确。

In [5]:
def generator_node(state):
    """Generates the initial draft of the code."""
    console.print("--- 1. Generating Initial Draft ---")
    generator_llm = llm.with_structured_output(DraftCode)
    
    prompt = f"""You are an expert Python programmer. Write a Python function to solve the following request.
    Provide a simple, clear implementation and an explanation.
    
    Request: {state['user_request']}
    """
    
    draft = generator_llm.invoke(prompt)
    return {"draft": draft.model_dump()} # Corrected: use .model_dump()

### 步骤 1.4：创建 Critic 节点

**我们要做什么：**
这是反思流程的核心。Critic 节点接收初稿，分析潜在缺陷，并使用我们的 `Critique` Pydantic 模型生成结构化的评审结果。

In [6]:
def critic_node(state):
    """Critiques the generated code for errors and inefficiencies."""
    console.print("--- 2. Critiquing Draft ---")
    critic_llm = llm.with_structured_output(Critique)
    
    code_to_critique = state['draft']['code']
    
    prompt = f"""You are an expert code reviewer and senior Python developer. Your task is to perform a thorough critique of the following code.
    
    Analyze the code for:
    1.  **Bugs and Errors:** Are there any potential runtime errors, logical flaws, or edge cases that are not handled?
    2.  **Efficiency and Best Practices:** Is this the most efficient way to solve the problem? Does it follow standard Python conventions (PEP 8)?
    
    Provide a structured critique with specific, actionable suggestions.
    
    Code to Review:
    ```python
    {code_to_critique}
    ```
    """
    
    critique = critic_llm.invoke(prompt)
    return {"critique": critique.model_dump()} # Corrected: use .model_dump()

### 步骤 1.5：创建 Refiner 节点

**我们要做什么：**
逻辑的最后一步是 Refiner。该节点接收原始草稿与结构化评审，并据此写出最终的改进版本。

In [7]:
def refiner_node(state):
    """Refines the code based on the critique."""
    console.print("--- 3. Refining Code ---")
    refiner_llm = llm.with_structured_output(RefinedCode)
    
    draft_code = state['draft']['code']
    critique_suggestions = json.dumps(state['critique'], indent=2)
    
    prompt = f"""You are an expert Python programmer tasked with refining a piece of code based on a critique.
    
    Your goal is to rewrite the original code, implementing all the suggested improvements from the critique.
    
    **Original Code:**
    ```python
    {draft_code}
    ```
    
    **Critique and Suggestions:**
    {critique_suggestions}
    
    Please provide the final, refined code and a summary of the changes you made.
    """
    
    refined_code = refiner_llm.invoke(prompt)
    return {"refined_code": refined_code.model_dump()} # Corrected: use .model_dump()

**Phase 1 讨论：**
我们已经创建了反思型 agent 的三个核心逻辑组件。每个组件都是执行单一且明确任务的函数（或 "节点"）。在各阶段使用结构化输出确保数据在节点之间可靠流转。现在，我们将用 LangGraph 对该工作流进行编排。

## Phase 2：用 LangGraph 编排反思工作流

### 步骤 2.1：定义图状态（Graph State）

**我们要做什么：**
"state" 是我们图的记忆。它是一个在节点之间传递的中心对象，每个节点都可以从其中读取或向其中写入。我们将用 Python 的 `TypedDict` 定义 `ReflectionState` 来容纳我们工作流的所有组成部分。

In [8]:
class ReflectionState(TypedDict):
    """Represents the state of our reflection graph."""
    user_request: str
    draft: Optional[dict]
    critique: Optional[dict]
    refined_code: Optional[dict]

print("ReflectionState TypedDict defined.")

ReflectionState TypedDict defined.


### 步骤 2.2：构建并可视化图

**我们要做什么：**
现在我们将使用 `StateGraph` 把各节点组装为一致的工作流。对于反思模式，工作流是简单的线性序列：**Generate → Critique → Refine**。我们会定义该流程，随后编译并尝试可视化图，以确认其结构。

In [9]:
graph_builder = StateGraph(ReflectionState)

# Add the nodes to the graph
graph_builder.add_node("generator", generator_node)
graph_builder.add_node("critic", critic_node)
graph_builder.add_node("refiner", refiner_node)

# Define the workflow edges
graph_builder.set_entry_point("generator")
graph_builder.add_edge("generator", "critic")
graph_builder.add_edge("critic", "refiner")
graph_builder.add_edge("refiner", END)

# Compile the graph
reflection_app = graph_builder.compile()

print("Reflection graph compiled successfully!")

# Visualize the graph
try:
    from IPython.display import Image, display
    png_image = reflection_app.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}. Please ensure pygraphviz is installed.")

Reflection graph compiled successfully!
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed.


**输出讨论：**
图已成功编译。可视化结果（若安装了 `pygraphviz`）可确认我们预期的线性工作流。你可以看到数据从入口（`generator`）经由 `critic` 与 `refiner` 节点，最后到达 `__end__`。这套简单而强大的结构现已准备好运行。

## Phase 3：端到端执行与评估

随着图被编译完成，接下来让我们在一个合适的编码任务上试试该反思模式。我们会选择一个首版实现容易不够优的案例，以便检验自我批判与改进的价值。

### 步骤 3.1：运行完整反思工作流

**我们要做什么：**
我们将用请求 "编写一个 Python 函数求第 n 个 Fibonacci 数" 来调用已编译的 LangGraph 应用。通过流式获取并累积最终 state，以便在结束时详细检查各个中间阶段。

In [10]:
user_request = "Write a Python function to find the nth Fibonacci number."
initial_input = {"user_request": user_request}

console.print(f"[bold cyan]🚀 Kicking off Reflection workflow for request:[/bold cyan] '{user_request}'\n")

# Corrected: This loop correctly captures the final, fully-populated state
final_state = None
for state_update in reflection_app.stream(initial_input, stream_mode="values"):
    final_state = state_update

console.print("\n[bold green]✅ Reflection workflow complete![/bold green]")

🚀 Kicking off Reflection workflow for request: 'Write a Python function to find the nth Fibonacci number.'

--- 1. Generating Initial Draft ---

--- 2. Critiquing Draft ---

--- 3. Refining Code ---

✅ Reflection workflow complete!

### 步骤 3.2：对“前后版本”进行分析

**我们要做什么：**
现在是检验时刻。我们将查看存储在 `final_state` 中的各阶段输出：打印初稿、它收到的评审，以及最终的改进代码，以便清晰地看到反思过程所带来的价值。

In [11]:
# Check if final_state is available and has the expected keys
if final_state and 'draft' in final_state and 'critique' in final_state and 'refined_code' in final_state:
    console.print(Markdown("--- ### Initial Draft ---"))
    console.print(Markdown(f"**Explanation:** {final_state['draft']['explanation']}"))
    # Use rich's Syntax for proper code highlighting
    console.print(Syntax(final_state['draft']['code'], "python", theme="monokai", line_numbers=True))

    console.print(Markdown("\n--- ### Critique ---"))
    console.print(Markdown(f"**Summary:** {final_state['critique']['critique_summary']}"))
    console.print(Markdown(f"**Improvements Suggested:**"))
    for improvement in final_state['critique']['suggested_improvements']:
        console.print(Markdown(f"- {improvement}"))

    console.print(Markdown("\n--- ### Final Refined Code ---"))
    console.print(Markdown(f"**Refinement Summary:** {final_state['refined_code']['refinement_summary']}"))
    console.print(Syntax(final_state['refined_code']['refined_code'], "python", theme="monokai", line_numbers=True))
else:
    console.print("[bold red]Error: The `final_state` is not available or is incomplete. Please check the execution of the previous cells.[/bold red]")

--- ### Initial Draft ---

Explanation: This function uses a recursive approach to calculate the nth Fibonacci number. It works by calling    
itself to calculate the (n-1)th and (n-2)th Fibonacci numbers and then adding them together. This approach is not  
efficient for large values of n due to the repeated calculations, but it is simple to understand and implement.

--- ### Critique ---

Summary: The function has potential bugs and inefficiencies. It should be revised to handle negative inputs and    
improve its time complexity.

Improvements Suggested:

**输出讨论：**
这些结果很好地展示了反思的力量。

1.  **Initial Draft** 通常会给出一个简单的递归实现。它在逻辑上是正确的，但由于反复计算相同的值，时间复杂度呈指数级，因此在规模较大时效率很差。
2.  **Critique** 能够识别出这一主要问题。在 "critic" 角色下，LLM 指出该实现的低效，并建议采用更优的迭代方式以避免冗余计算。
3.  **Final Refined Code** 落实了评审意见：使用循环和两个变量跟踪序列，替代了缓慢的递归方案。 

这不是修补一个小错误，而是对算法进行实质性改进。agent 并非只是改个拼写，而是将解法提升为更稳健、可扩展的版本。这就是反思模式的价值。

### 步骤 3.3：定量评估（LLM-as-a-Judge）

**我们要做什么：**
为让分析更正式，我们将用另一位 LLM 作为不偏不倚的 "judge"，对初稿与最终代码的质量进行打分。这为反思带来的改进提供更客观的量化依据。

In [12]:
class CodeEvaluation(BaseModel):
    """Schema for evaluating a piece of code."""
    correctness_score: int = Field(description="Score from 1-10 on whether the code is logically correct.")
    efficiency_score: int = Field(description="Score from 1-10 on the code's algorithmic efficiency.")
    style_score: int = Field(description="Score from 1-10 on code style and readability (PEP 8). ")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(CodeEvaluation)

def evaluate_code(code_to_evaluate: str):
    prompt = f"""You are an expert judge of Python code. Evaluate the following function on a scale of 1-10 for correctness, efficiency, and style. Provide a brief justification.
    
    Code:
    ```python
    {code_to_evaluate}
    ```
    """
    return judge_llm.invoke(prompt)

if final_state and 'draft' in final_state and 'refined_code' in final_state:
    console.print("--- Evaluating Initial Draft ---")
    initial_draft_evaluation = evaluate_code(final_state['draft']['code'])
    console.print(initial_draft_evaluation.model_dump()) # Corrected: use .model_dump()

    console.print("\n--- Evaluating Refined Code ---")
    refined_code_evaluation = evaluate_code(final_state['refined_code']['refined_code'])
    console.print(refined_code_evaluation.model_dump()) # Corrected: use .model_dump()
else:
    console.print("[bold red]Error: Cannot perform evaluation because the `final_state` is incomplete.[/bold red]")

--- Evaluating Initial Draft ---

{
    'correctness_score': 2,
    'efficiency_score': 2,
    'style_score': 2,
    'justification': 'The function has a time complexity of O(2^n) due to the repeated computation of the same 
    subproblems, which makes it inefficient. The code also does not follow the PEP 8 style guide as it does not have a 
    consistent indentation and the function name does not follow the conventional snake_case naming convention. The 
    base cases are also not handled correctly, as the function returns 0 for n <= 0, but the Fibonacci sequence is 
    typically defined as 0, 1, 1, 2, 3, 5, 8, 13, ...'
}

--- Evaluating Refined Code ---

{
    'correctness_score': 8,
    'efficiency_score': 6,
    'style_score': 9,
    'justification': 'The code is correct for calculating the nth Fibonacci number. However, it has a time 
    complexity of O(n) due to the use of a loop, which is not efficient for large values of n. The style is good, 
    following PEP 8 conventions, but it could be improved by using a more efficient algorithm like memoization or 
    dynamic programming.'
}

**输出讨论：**
LLM-as-a-Judge 的评分为反思模式的成功提供了定量证据。初稿在正确性方面或许得分不低，但在效率方面会很低；而最终代码则在正确性与效率上都能获得较高分。该自动化评分说明反思过程不只是改变了代码，而且在可度量的层面上确实实现了“改进”。

## 总结

在本 notebook 中，我们使用 Nebius AI Studio 模型构建并运行了完整的 **Reflection** 架构，并对其进行了评估。我们亲眼看到，这一简单却强大的模式如何让一个基础的 LLM 生成器，转变为更加成熟可靠的问题求解者。

通过将流程结构化为 **Generate**、**Critique**、**Refine** 三步，并用 LangGraph 进行编排，我们构建了一个能够识别并纠正重大缺陷的系统。从低效的递归解到更优的迭代实现的实质性提升，表明 Reflection 是从“琐碎 agentic 任务”迈向“更高质量与更深入审慎”的基础技术。